# Introduction to Microtonal Music Making through the Tonality Diamond

This notebook introduces several concepts and realizes music using those concepts
- Intro to the Tonality Diamond to the 31-limit
- Creating some structures that will assist in making music
- building the csound orchestra
- making some music
- adding some glissandos and trills

## Introduction to the Tonality Diamond

Harry Partch was a microtonal theoretician, composer, instrument builder, sometimes hobo, and all around genius. His book, Genesis of a Music, is a guide to his musical ideas. The wiki page is helpful guide to the book: __[wiki page](https://en.wikipedia.org/wiki/Genesis_of_a_Music)__

This notebook carries on with one of his ideas, namely the tonality diamond. I've been playing around with tuning systems derived from this concept for many years. The basic idea is that you calculate the overtones of a root note, in Partch's case 'G' to some arbitrary limit. Partch's "Incipient Tonality Diamond" took the overtones to the 5 limit. Partch further postulated what he called undertones, the inverse of an overtone, to the same limit based on notes in the overtone series. This gave him a structure of overtones on the undertones, and overtones on the undertones. Here is an image of the 5-limit diamond, with unison in the middle from top to bottom (3:3, 5:5, and 1:1), overtones going up and to the right (1:1, 5:4 3:2) and undertones on those going up and to the left (1:1, 8:5, 4:3).

<img src='http://tonalsoft.com/enc/t/images/diamond.gif'>

He quickly expanded to the 11-limit to find more tones. I continued with this exploration with pieces in the 15-limit, and now the 31-limit. As the limit rises, the number of tones per octave also increase. Which leads to the sizing exercise in the next few cells.

## Size of the Tonality Diamond

Wikipedia: https://en.wikipedia.org/wiki/Tonality_diamond 

According to wiki, there is a direct way to calculate how many unique pitches in a tonality diamond. <blockquote>
    If φ(n) is Euler's totient function, which gives the number of positive integers less than n and relatively prime to n, that is, it counts the integers less than n which share no common factor with n, and if d(n) denotes the size of the n-limit tonality diamond, we have the formula.
</blockquote>

They go on to describe Euler's Totient:
<img src='http://ripnread.com/listen/EulerTotientFunction.svg'>
Again according to wiki, there are 
- 7 members to the 5-limit diamond, 
- 13 to the 7-limit diamond, 
- 19 to the 9-limit diamond, 
- 29 to the 11-limit diamond, 
- 41 to the 13-limit diamond, and 
- 49 to the 15-limit diamond; 
And they further state that "that these suffice for most purposes."

I disagree, and have been using the tonality diamond to the 31 limit for about five years or so.

First let's load all the functions I might need

In [1]:
import numpy as np
import numpy.testing as npt
import pprint
pp = pprint.PrettyPrinter(sort_dicts=False)
import copy
import mido
import time
from importlib import reload
from IPython.display import Audio, display
import os
import muspy
import pandas as pd
import sys
import logging
import ctcsound
# sys.path.insert(0, '/home/prent/Dropbox/Tutorials/coconet-pytorch/coconet-pytorch-csound')
# import piano as p
# import selective_stretching_codes as stretch
# import samples_used as su
import subprocess
from numpy.random import default_rng
rng = np.random.default_rng()
from fractions import Fraction
CSD_FILE = 'ball3.csd'
LOGNAME = 'ball3.log'
WAV_OUT = '/home/prent/Music/sflib/ball3.wav'
CON_OUT = '/home/prent/Music/sflib/ball3-t'

OCTAVE_SIZE = 256 # there are 213 distinct pictches per octave in the tonality diamond to the 31-limit, but the array storing cents is 256
first_item_only = True # limits the output to the first few of a set
csound_running = False # lets me know if we already have an instance of csound available to ctcsound

In [2]:
# brute force method to calculate the size of a tonality diamond

for limit in (5, 7, 9, 11, 13, 15, 31): # calculate the size of a variety of standard limits, including 31-limit
    end_denom = limit + 1
    start_denom = (end_denom) // 2
    o_numerator = np.arange(start_denom, end_denom, 1) # create a list of overtones
    u_denominator = np.arange(start_denom, end_denom, 1) # create a list of undertones
    print(f'Tonality Diamond in the Cassandra orientation to the {limit}-limit', end='')
    all_ratios = []
    for oton_root in u_denominator:
        print()
        for overtone in o_numerator:
            if overtone < oton_root: oton = overtone * 2
            else: oton = overtone
            print(f'{str(Fraction(oton / oton_root).limit_denominator())}', end = '\t')
            all_ratios.append(oton / oton_root)

    no_dupes = list(dict.fromkeys(all_ratios)) # eliminate duplicates
    
    print(f'\nThere are {len(no_dupes)} unique members in the {limit}-limit diamond\n\n')      

Tonality Diamond in the Cassandra orientation to the 5-limit
1	4/3	5/3	
3/2	1	5/4	
6/5	8/5	1	
There are 7 unique members in the 5-limit diamond


Tonality Diamond in the Cassandra orientation to the 7-limit
1	5/4	3/2	7/4	
8/5	1	6/5	7/5	
4/3	5/3	1	7/6	
8/7	10/7	12/7	1	
There are 13 unique members in the 7-limit diamond


Tonality Diamond in the Cassandra orientation to the 9-limit
1	6/5	7/5	8/5	9/5	
5/3	1	7/6	4/3	3/2	
10/7	12/7	1	8/7	9/7	
5/4	3/2	7/4	1	9/8	
10/9	4/3	14/9	16/9	1	
There are 19 unique members in the 9-limit diamond


Tonality Diamond in the Cassandra orientation to the 11-limit
1	7/6	4/3	3/2	5/3	11/6	
12/7	1	8/7	9/7	10/7	11/7	
3/2	7/4	1	9/8	5/4	11/8	
4/3	14/9	16/9	1	10/9	11/9	
6/5	7/5	8/5	9/5	1	11/10	
12/11	14/11	16/11	18/11	20/11	1	
There are 29 unique members in the 11-limit diamond


Tonality Diamond in the Cassandra orientation to the 13-limit
1	8/7	9/7	10/7	11/7	12/7	13/7	
7/4	1	9/8	5/4	11/8	3/2	13/8	
14/9	16/9	1	10/9	11/9	4/3	13/9	
7/5	8/5	9/5	1	11/10	6/5	13/10	
14/1

Here is an image of the cassandra representation of the 31-limit tonality diamond including the note names with Sagittal accidentals and the index into the 256-member array of notes. The index is used to pull cent values from a table when using Csound. They don't reflect the relative pitch, just the index into a 256 member array.
The overtone series runs from left to right, and the undertone from top to bottom.
![31-limit Cassandra](31-limit_cassandra.jpg)

In [3]:
# We now have an list with all the ratios in the diamond to the 31-limit in all_ratios. 
# This list does not have duplicates removed. I don't mind having a few extra of the same ratio.
print(len(all_ratios))

256


In the skeleton csound file provided, we need these ratios as values that the csound function cpspch can convert to frequency in Hertz.

Here is the csound code that converts a note in the Tonality Diamond to the 31-limit into fractions of an octave, that csound can turn into a frequency that can be performed.
<code>
   ipitch table p5, 3 ; convert note number 0-256 to oct.fract format in a table of values
   kcps = cpspch(ioct + ipitch) ; convert oct.fract to Hz at krate 
</code>
<p>To build that table, we need these ratios converted into cents, then divide that by 10,000. The formula for converting a ratio into cents is shown in use in the next cell.

In [4]:
# conversion between ratios (just) and cents (1200 equal per octave)
ratio = 51/50
print(f'convert the ratio {str(Fraction(ratio).limit_denominator(max_denominator = 100))} to cents: {round(1200 * np.log(ratio)/np.log(2),1)}')
# cents to ratio
cents = 34.3
print(f'convert {cents} cents to a ratio: {str(Fraction(np.power(2, cents/1200)).limit_denominator(max_denominator = 100))}')

convert the ratio 51/50 to cents: 34.3
convert 34.3 cents to a ratio: 51/50


In [56]:
# let's take a look at the list of ratios, converted into ratios we can see:
# first five ratios
print(f'first five ratios: {[str(Fraction(ratio).limit_denominator(max_denominator = 100)) for ratio in all_ratios[:5]] }')
# last five ratios
print(f'last five ratios: {[str(Fraction(ratio).limit_denominator(max_denominator = 100)) for ratio in all_ratios[-5:]] }')

first five ratios: ['1', '17/16', '9/8', '19/16', '5/4']
last five ratios: ['54/31', '56/31', '58/31', '60/31', '1']


In [57]:
print(f'first five fractions that csound needs: {[1200 * np.log(ratio)/np.log(2)/10_000 for ratio in all_ratios[:5]]}')
print(f'last five fractions that csound needs: {[1200 * np.log(ratio)/np.log(2)/10_000 for ratio in all_ratios[-5:]]}')

first five fractions that csound needs: [0.0, 0.010495540950040728, 0.020391000173077486, 0.02975130161323026, 0.03863137138648348]
last five fractions that csound needs: [0.0960829430131912, 0.10237903340048746, 0.10845416216888361, 0.11432331422659722, 0.0]


In [58]:
temp_array = np.array(all_ratios, dtype = float)
print(f'{temp_array[0:5] = }')
t_array = np.array([i for i in temp_array])
print(f'{t_array[0:5] = }')

temp_array[0:5] = array([1.    , 1.0625, 1.125 , 1.1875, 1.25  ])
t_array[0:5] = array([1.    , 1.0625, 1.125 , 1.1875, 1.25  ])


In [59]:
f3_array = np.array([1200 * np.log(ratio)/np.log(2)/10_000 for ratio in all_ratios]) # convert it into a numpy array

In [60]:
print(f'first 25 values {[fract for fract in f3_array[:25]]}')

first 25 values [0.0, 0.010495540950040728, 0.020391000173077486, 0.02975130161323026, 0.03863137138648348, 0.04707809073345123, 0.05513179423647566, 0.06282743472684155, 0.07019550008653874, 0.07726274277296696, 0.08405276617693107, 0.09058650025961625, 0.09688259064691249, 0.10295771941530865, 0.10882687147302222, 0.11450355724642503, 0.10950445904995929, 0.0, 0.009895459223036756, 0.019255760663189535, 0.028135830436442757, 0.036582549783410516, 0.04463625328643495, 0.05233189377680082, 0.059699959136498025]


In [10]:
# to load a function table, we have to give csound some information about it in a preface:
#                      +-- the function table number we are creating (3)
#                      |  +-- 0 start immediately
#                      |  |  +-- the size of the array (prefers a power of two)
#                      |  |  |     +-- the type of table format, in this case values that will serve as a lookup table indexed by the note number
#                      |  |  |     |   that returns an octave fraction to use to build a frequency in Hertz. negative to supress normalization
#                      |  |  |     |     
f3_preface = np.array([3, 0, 256, -2])

f3_array_ready_to_load = np.concatenate((f3_preface,f3_array)) 
# now it is ready to pass to csound. 

In [222]:
print(f'first 6 values {[fract for fract in f3_array_ready_to_load[:6]]}') # should be [3.0, 0.0, 256.0, -2.0, 0.0, 0.010495540950040728]
print(f'last 3 values {[fract for fract in f3_array_ready_to_load[-3:]]}') # should be [0.11432331422659722, 0.11450355724642503, 0.12]

first 6 values [3.0, 0.0, 256.0, -2.0, 0.0, 0.010495540950040728]
last 3 values [0.11432331422659722, 0.0, 0.12]


In [223]:
# let's go back to our initial tonality diamond ratios.
# They are not sorted, neither were the fractions we are ready to load into csound 
# we need a way to look up fraction values in the csound table f3_array_ready_to_load by their position in the csound table

In [62]:
ratio_strings = np.array([str(Fraction(ratio).limit_denominator(max_denominator = 100)) for ratio in all_ratios]).reshape(16,16)
print(f'first 16 values will be the notes of the first otonal scale: {ratio_strings[0,:16] = }')

first 16 values will be the notes of the first otonal scale: ratio_strings[0,:16] = array(['1', '17/16', '9/8', '19/16', '5/4', '21/16', '11/8', '23/16',
       '3/2', '25/16', '13/8', '27/16', '7/4', '29/16', '15/8', '31/16'],
      dtype='<U5')


In [64]:
print(f'the first 15 values in all_ratios {[ratio for ratio in all_ratios[:15]]}')
print(f'first 15 values in f3_array_ready_to_load {[fract for fract in f3_array_ready_to_load[:25]]}') 

the first 15 values in all_ratios [1.0, 1.0625, 1.125, 1.1875, 1.25, 1.3125, 1.375, 1.4375, 1.5, 1.5625, 1.625, 1.6875, 1.75, 1.8125, 1.875]
first 15 values in f3_array_ready_to_load [3.0, 0.0, 256.0, -2.0, 0.0, 0.010495540950040728, 0.020391000173077486, 0.02975130161323026, 0.03863137138648348, 0.04707809073345123, 0.05513179423647566, 0.06282743472684155, 0.07019550008653874, 0.07726274277296696, 0.08405276617693107, 0.09058650025961625, 0.09688259064691249, 0.10295771941530865, 0.10882687147302222, 0.11450355724642503, 0.10950445904995929, 0.0, 0.009895459223036756, 0.019255760663189535, 0.028135830436442757]


In [65]:
# fix the ratio strings that are '1' to be '1/1' like all the other ratios
i = 0
for ratio in ratio_strings:
    j = 0
    for r in ratio:
        if ratio_strings[i,j] == '1': 
            ratio_strings[i,j] = '1/1'
        j += 1
    i += 1
print(f'{ratio_strings[0] = }')

ratio_strings[0] = array(['1/1', '17/16', '9/8', '19/16', '5/4', '21/16', '11/8', '23/16',
       '3/2', '25/16', '13/8', '27/16', '7/4', '29/16', '15/8', '31/16'],
      dtype='<U5')


In [ ]:
# build the chord structures for the different modes for the 16 keys
# build the 16 tone scales in each of the keys in the diamond. otonal first, then utonal
# nested dictionary keys[mode][ratio_name] 
keys = {'oton': {ratio_strings[0, 0]: np.arange(0 * 16, 1 * 16, 1),
                 ratio_strings[1 ,0]: np.arange(1 * 16, 2 * 16, 1),
                 ratio_strings[2 ,0]: np.arange(2 * 16, 3 * 16, 1),
                 ratio_strings[3 ,0]: np.arange(3 * 16, 4 * 16, 1),
                 ratio_strings[4 ,0]: np.arange(4 * 16, 5 * 16, 1),
                 ratio_strings[5 ,0]: np.arange(5 * 16, 6 * 16, 1),
                 ratio_strings[6 ,0]: np.arange(6 * 16, 7 * 16, 1),
                 ratio_strings[7 ,0]: np.arange(7 * 16, 8 * 16, 1),
                 ratio_strings[8 ,0]: np.arange(8 * 16, 9 * 16, 1),
                 ratio_strings[9 ,0]: np.arange(9 * 16, 10 * 16, 1),
                 ratio_strings[10, 0]: np.arange(10 * 16, 11 * 16, 1),
                 ratio_strings[11, 0]: np.arange(11 * 16, 12 * 16, 1),
                 ratio_strings[12, 0]: np.arange(12 * 16, 13 * 16, 1),
                 ratio_strings[13, 0]: np.arange(13 * 16, 14 * 16, 1),
                 ratio_strings[14, 0]: np.arange(14 * 16, 15 * 16, 1),
                 ratio_strings[15, 0]: np.arange(15 * 16, 16 * 16, 1)
                },
        'uton': {ratio_strings[0, 0]: np.arange(0, 256, 16),
                 ratio_strings[0, 1]: np.arange(1, 256, 16),
                 ratio_strings[0, 2]: np.arange(2, 256, 16),
                 ratio_strings[0, 3]: np.arange(3, 256, 16),
                 ratio_strings[0, 4]: np.arange(4, 256, 16),
                 ratio_strings[0, 5]: np.arange(5, 256, 16),
                 ratio_strings[0, 6]: np.arange(6, 256, 16),
                 ratio_strings[0, 7]: np.arange(7, 256, 16),
                 ratio_strings[0, 8]: np.arange(8, 256, 16),
                 ratio_strings[0, 9]: np.arange(9, 256, 16),
                 ratio_strings[0, 10]: np.arange(10, 256, 16),
                 ratio_strings[0, 11]: np.arange(11, 256, 16),
                 ratio_strings[0, 12]: np.arange(12, 256, 16),
                 ratio_strings[0, 13]: np.arange(13, 256, 16),
                 ratio_strings[0, 14]: np.arange(14, 256, 16),
                 ratio_strings[0, 15]: np.arange(15, 256, 16)                 
                }
                }

In [67]:
# with the keys() dictionary, we have a way to build 16 note sequences of indexes to note in the overtones or undertones. 
# We can find the ratios by accessing the converting those indeciis to ratio strings using the all_ratio_strings array
all_ratio_strings = ratio_strings.reshape(256,)
# here is an example of pulling an undertone scale sequence on the 17:16 overtone root
scale_index = keys["uton"]["17/16"] # 16 indeciis into the 256 member index array
print(f'scale degrees: {scale_index = }') # show the indeciis
print(f'ratios from scale_degrees: {[all_ratio_strings[degree] for degree in scale_index]}') # show the ratios 
print(f'All the uton roots: {keys["uton"].keys() = }') # what are the roots available in the undertone series
print(f'All the oton roots: {keys["oton"].keys() = }') # what are the roots available in the overtone series

for mode in keys.keys(): # 'oton', 'uton'
    for ratio in keys[mode]:
        print(f'{mode = }, {ratio = }, {keys[mode][ratio] = }')
        if first_item_only: break    

# check if a key is available in the mode before accessing it:
if '16/9' in keys['oton']: 
    scale_index = keys["oton"]["16/9"]
    print(f'scale degrees: {scale_index = }')
    print(f'ratios in scale_index: {[all_ratio_strings[degree] for degree in scale_index]}')

print(f'{all_ratio_strings[scale_index] = }')

scale degrees: scale_index = array([  1,  17,  33,  49,  65,  81,  97, 113, 129, 145, 161, 177, 193,
       209, 225, 241])
ratios from scale_degrees: ['17/16', '1/1', '17/9', '34/19', '17/10', '34/21', '17/11', '34/23', '17/12', '34/25', '17/13', '34/27', '17/14', '34/29', '17/15', '34/31']
All the uton roots: keys["uton"].keys() = dict_keys(['1/1', '17/16', '9/8', '19/16', '5/4', '21/16', '11/8', '23/16', '3/2', '25/16', '13/8', '27/16', '7/4', '29/16', '15/8', '31/16'])
All the oton roots: keys["oton"].keys() = dict_keys(['1/1', '32/17', '16/9', '32/19', '8/5', '32/21', '16/11', '32/23', '4/3', '32/25', '16/13', '32/27', '8/7', '32/29', '16/15', '32/31'])
mode = 'oton', ratio = '1/1', keys[mode][ratio] = array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15])
mode = 'uton', ratio = '1/1', keys[mode][ratio] = array([  0,  16,  32,  48,  64,  80,  96, 112, 128, 144, 160, 176, 192,
       208, 224, 240])
scale degrees: scale_index = array([32, 33, 34, 35, 36, 37, 38, 39

## Scale alternatives in the Tonality Diamond to the 31-limit

It can be challenging to approach a scale with 213 distinct pitches. I divide the scale into different sets. The first of these is the mode. The 'oton' mode is for the overtone series. The 'uton' mode is for the inverse, the undertone series. It is possible to create a very strong major scale out of the 16 notes of the 'oton' mode. But there are equally interesting major scales to be found elsewhere in both the 'oton' mode and the 'uton' mode. Each has a different character. There are also several interesting minor scales in the 'oton' mode.

The same is true with the 'uton' mode. There is a strong minor scale made by starting at the root of the undertone series and descending down a 3:2 from there. And there are a few interesting major scales and some odd minor scales in the 'uton' mode. 

A Rank is an 8 note scale derived from the 16 notes in the 'oton' or 'uton' mode. 

- Rank A is made up of overtones 8, 9, 10, 11, 12, 13, 14, 15/16 of each 'oton' or 16/15, 14, 13, 12, 11, 10, 9, 8 of the 'uton' mode
- Rank B is made up of overtones 9, 10, 11, 12, 13, 14, 15, 16/16 of each 'oton' or 16/16, 15, 14, 13, 12, 11, 10, 9 of the 'uton' mode
- Rank C is made up of overtones 17, 19, 11, 23, 25, 27, 29, 31/32 of each otonal ratio or 32/31, 29, 27, 25, 23, 21, 19, 17 of the 'uton' 
- Rank D is made up of overtones 19, 11, 23, 25, 27, 29, 31, 17/32 of each otonal ratio or 32/17, 31, 29, 27, 25, 23, 21, 19 of the 'uton'
- Rank E through H are additional 8-note scales that each has a perfect 3:2 and a variety of third degrees. These two intervals are key to the mood of a scale.

Ranks A & B are made up a tonality diamond to the 15-limit. Ranks C & D take it to the 31-limit. Ranks E through H pull from both 15 and 31-limit sources. A simple switch of ranks creates drastically different moods. 

The next cell creates an array called 'scales' that simplifies the process of pulling those scale ranks out of the 31-limit diamond.

In [68]:
# build the 8 note scales for each of the rank A, B, C, D otonal and utonal out of the 16 note scales
#                                                        start, end, step size
# make this dictionary nested: rank, mode
scales = {'A': {'oton': np.array([note % 16 for note in np.arange(0, 16, 2)]),
                'uton': np.array([note % 16 for note in np.arange(8, -8, -2)])}, # utonal goes down, so that the scale will go up.
          'B': {'oton': np.array([note % 16 for note in np.arange(2, 18, 2)]),
                'uton': np.array([note % 16 for note in np.arange(14, -2, -2)])},
          'C': {'oton': np.array([note % 16 for note in np.arange(1, 17, 2)]),
                'uton': np.array([note % 16 for note in np.arange(13, -2, -2)])},
          'D': {'oton': np.array([note % 16 for note in np.arange(3, 19, 2)]),
                'uton': np.array([note % 16 for note in np.arange(15, 0, -2)])},
          # the next 8 are additional 8 note scales that have good 3:2, and interesting thirds
          'E': {'oton': np.array([ 2,  4,  6,  8, 11, 13, 15,  0]), # 3rd: 11:9 neutral
                'uton': np.array([ 8,  6,  4,  2,  0, 15, 13, 11])}, # 3rd: 6:5 minor
          'F': {'oton': np.array([ 8, 10, 12, 14,  0,  2,  4,  6]), # 3rd: 7:6 subminor
                'uton': np.array([ 0, 14, 12, 10,  8,  6,  4,  2])}, # 3rd: 8:7 subminor
          'G': {'oton': np.array([ 4,  6,  8, 10, 14, 15,  0,  2]), # 3rd: 6:5 minor
                'uton': np.array([14, 10,  8,  6,  4,  2,  0, 15])}, # 3rd: 5:4 major
          'H': {'oton': np.array([12, 14,  0,  2,  5,  7,  9, 11]), # 3rd: 8:7 sub-subminor
                'uton': np.array([5,  3,   1, 15, 12, 10,  8,  6])} # 3rd: 21/17 neutral
               }
pp.pprint(scales)
print()
print(f'{scales.keys() = }')

{'A': {'oton': array([ 0,  2,  4,  6,  8, 10, 12, 14]),
       'uton': array([ 8,  6,  4,  2,  0, 14, 12, 10])},
 'B': {'oton': array([ 2,  4,  6,  8, 10, 12, 14,  0]),
       'uton': array([14, 12, 10,  8,  6,  4,  2,  0])},
 'C': {'oton': array([ 1,  3,  5,  7,  9, 11, 13, 15]),
       'uton': array([13, 11,  9,  7,  5,  3,  1, 15])},
 'D': {'oton': array([ 3,  5,  7,  9, 11, 13, 15,  1]),
       'uton': array([15, 13, 11,  9,  7,  5,  3,  1])},
 'E': {'oton': array([ 2,  4,  6,  8, 11, 13, 15,  0]),
       'uton': array([ 8,  6,  4,  2,  0, 15, 13, 11])},
 'F': {'oton': array([ 8, 10, 12, 14,  0,  2,  4,  6]),
       'uton': array([ 0, 14, 12, 10,  8,  6,  4,  2])},
 'G': {'oton': array([ 4,  6,  8, 10, 14, 15,  0,  2]),
       'uton': array([14, 10,  8,  6,  4,  2,  0, 15])},
 'H': {'oton': array([12, 14,  0,  2,  5,  7,  9, 11]),
       'uton': array([ 5,  3,  1, 15, 12, 10,  8,  6])}}

scales.keys() = dict_keys(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H'])


In [69]:
# this dictionary is helpful in doing a lookup of different inversions of a chord
# choose the notes in scale for each rank (A, B, C, D), mode (oton, uton), & inversion (1, 2, 3, 4)
# access the four note chords by specifying inversions[rank][mode][inv]
# where rank is in (A, B, C, D) mode is in (oton, uton), and inv is in (1, 2, 3, 4)
# use the resulting array as the index into a keys[mode][ratio]

inversions = {'A': {'oton': {1: np.array([0, 4, 8, 12]),
                             2: np.array([4, 8, 12, 0]),
                             3: np.array([8, 12, 0, 4]),
                             4: np.array([12, 0, 4, 8])},
                    'uton': {1: np.flip(np.array([0, 4, 8, 12])),
                             2: np.flip(np.array([4, 8, 12, 0])),
                             3: np.flip(np.array([8, 12, 0, 4])),
                             4: np.flip(np.array([12, 0, 4, 8]))}}, 
              'B': {'oton': {1: np.array([2, 6, 10, 14]),
                             2: np.array([6, 10, 14, 2]),
                             3: np.array([10, 14, 2, 6]),
                             4: np.array([14, 2, 6, 10])},
                    'uton': {1: np.flip(np.array([2, 6, 10, 14])),
                             2: np.flip(np.array([6, 10, 14, 2])),
                             3: np.flip(np.array([10, 14, 2, 6])),
                             4: np.flip(np.array([14, 2, 6, 10]))}}, 
              'C': {'oton': {1: np.array([1, 5, 9, 13]),
                             2: np.array([5, 9, 13, 1]),
                             3: np.array([9, 13, 1, 5]),
                             4: np.array([13, 1, 5, 9])},
                    'uton': {1: np.flip(np.array([2, 6, 10, 14])),
                             2: np.flip(np.array([6, 10, 14, 2])),
                             3: np.flip(np.array([10, 14, 2, 6])),
                             4: np.flip(np.array([14, 2, 6, 10]))}}, 
              'D': {'oton': {1: np.array([3, 7, 11, 15]),
                             2: np.array([7, 11, 15, 3]),
                             3: np.array([11, 15, 3, 7]),
                             4: np.array([15, 3, 7, 11])},
                    'uton': {1: np.flip(np.array([3, 7, 11, 15])),
                             2: np.flip(np.array([7, 11, 15, 3])),
                             3: np.flip(np.array([11, 15, 3, 7])),
                             4: np.flip(np.array([15, 3, 7, 11]))}},
              'E': {'oton': {1: np.array([ 2,  6, 11, 15]),
                             2: np.array([ 6, 11, 15,  2]),
                             3: np.array([11, 15,  2,  6]),
                             4: np.array([15,  2,  6, 11])},
                    'uton': {1: np.array([8,  4,  0, 13]),
                             2: np.array([4,  0, 13,  8]),
                             3: np.array([0, 13,  8,  4]),
                             4: np.array([8,  4,  0, 13])}},
              'F': {'oton': {1: np.array([ 8, 12,  0,  4]),
                             2: np.array([12, 0,  4, 8]),
                             3: np.array([ 0, 4, 8, 12]),
                             4: np.array([ 4, 8, 12,  0])},
                    'uton': {1: np.array([ 0, 12,  8,  2]),
                             2: np.array([12,  8,  2,  0]),
                             3: np.array([ 8,  2,  0, 12]),
                             4: np.array([ 2,  0, 12,  8])}},
              'G': {'oton': {1: np.array([ 4,  8, 14,  0]),
                             2: np.array([ 8, 14,  0,  4]),
                             3: np.array([14,  0,  4,  8]),
                             4: np.array([ 0,  4,  8, 14])},
                    'uton': {1: np.array([14,  8,  4,  0]),
                             2: np.array([ 8,  4,  0, 14]),
                             3: np.array([ 4,  0, 14,  8]),
                             4: np.array([ 0, 14,  8,  4])}},
              'H': {'oton': {1: np.array([12,  0,  5,  9]),
                             2: np.array([ 0,  5,  9, 12]),
                             3: np.array([ 5,  9, 12,  0]),
                             4: np.array([ 9, 12,  0,  5])},
                    'uton': {1: np.array([ 5,  1, 12,  8]),
                             2: np.array([ 1, 12,  8,  5]),
                             3: np.array([12,  8,  5,  1]),
                             4: np.array([ 8,  5,  1, 12])}}}
             

print(f'{inversions.keys() = }')
for rank in inversions.keys():
    print(f'{rank = }')
    # print(f'{rank = }: {[inversions[rank][mode] for mode in inversions[rank].keys()]}')
    for mode in inversions[rank]:
        print(f'{mode = }')
        for chord_inversion in inversions[rank][mode]:
            print(f'inversion: {chord_inversion}: {inversions[rank][mode][chord_inversion] = }')
    if first_item_only: break

inversions.keys() = dict_keys(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H'])
rank = 'A'
mode = 'oton'
inversion: 1: inversions[rank][mode][chord_inversion] = array([ 0,  4,  8, 12])
inversion: 2: inversions[rank][mode][chord_inversion] = array([ 4,  8, 12,  0])
inversion: 3: inversions[rank][mode][chord_inversion] = array([ 8, 12,  0,  4])
inversion: 4: inversions[rank][mode][chord_inversion] = array([12,  0,  4,  8])
mode = 'uton'
inversion: 1: inversions[rank][mode][chord_inversion] = array([12,  8,  4,  0])
inversion: 2: inversions[rank][mode][chord_inversion] = array([ 0, 12,  8,  4])
inversion: 3: inversions[rank][mode][chord_inversion] = array([ 4,  0, 12,  8])
inversion: 4: inversions[rank][mode][chord_inversion] = array([ 8,  4,  0, 12])


In [70]:
# Is there anything else I need to track about the voice? If not, this could be simplified
voice_time = {'fpn': {'full': 'finger piano', 'start': 0, 'number': 1},
 'bnp': {'full': 'bass finger piano', 'start': 0, 'number': 2},
 'bdl': {'full': 'bass balloon drum', 'start': 0, 'number': 3},
 'bdm': {'full': 'medium balloon drum', 'start': 0, 'number': 4},
 'bdh': {'full': 'high balloon drum', 'start': 0, 'number': 5},
 'bfl': {'full': 'bass flute', 'start': 0, 'number': 6},
 'obo': {'full': 'oboe', 'start': 0, 'number': 7},
 'cla': {'full': 'clarinet', 'start': 0, 'number': 8},
 'bss': {'full': 'bassoon', 'start': 0, 'number': 9},
 'frn': {'full': 'french horn', 'start': 0, 'number': 10}}

pp.pprint(voice_time)
print(f'{voice_time["fpn"]["full"] = }')
print(f'{voice_time["fpn"]["start"] = }')
print(f'{voice_time["fpn"]["number"] = }')

{'fpn': {'full': 'finger piano', 'start': 0, 'number': 1},
 'bnp': {'full': 'bass finger piano', 'start': 0, 'number': 2},
 'bdl': {'full': 'bass balloon drum', 'start': 0, 'number': 3},
 'bdm': {'full': 'medium balloon drum', 'start': 0, 'number': 4},
 'bdh': {'full': 'high balloon drum', 'start': 0, 'number': 5},
 'bfl': {'full': 'bass flute', 'start': 0, 'number': 6},
 'obo': {'full': 'oboe', 'start': 0, 'number': 7},
 'cla': {'full': 'clarinet', 'start': 0, 'number': 8},
 'bss': {'full': 'bassoon', 'start': 0, 'number': 9},
 'frn': {'full': 'french horn', 'start': 0, 'number': 10}}
voice_time["fpn"]["full"] = 'finger piano'
voice_time["fpn"]["start"] = 0
voice_time["fpn"]["number"] = 1


## Create the helper functions to assemble notes and chords
Each of these functions provide some capability that simplifies the creation of music to some degree. At some point, once I've had some experience with them, I'll move them in to a python library to reduce the size of the notebook.

In [71]:
pp.pprint(voice_time)
print(f'{voice_time["fpn"]["full"] = }')
print(f'{voice_time["fpn"]["start"] = }')
print(f'{voice_time["fpn"]["number"] = }')

In [72]:
def build_scales(mode, ratio, rank):
    # print(f'{mode = }, {ratio = }, {rank = }')
    # if rank in (['A', 'B', 'C', 'D']):
    if ratio in keys[mode]: 
        scale = np.array([keys[mode][ratio][note] for note in (scales[rank][mode])])
    else: 
        print(f'in build_scales. Could not find {ratio = } with {mode} in {keys[mode] = } with {rank = }')
        scale = None
    return scale

In [73]:
def ratio_string_to_float(ratio):
    n, d = ratio.split('/')
    return float(float(n) / float(d))

In [221]:
# note that this function tries to find the closest end note to the start note. It might be up or down, it might cross an octave boundary.
# for example, from 15:8 to 1:1 would ordinarily return 15:8 distance between the two notes, but the closest one is from 15:8 to 2:1, moving up an octave, multiplying the second ratio by 2
# to accomodate that, the function tests the size of the ratio returned from the normal calculation, and multiplies either the start or the end note by 2 before recalculating the ratio.
# this can cause problems if you are looking for that large leap. You can override the default behavior by setting find_closest = False
def ratio_distance(start, end, find_closest = True):
    start_ratio = ratio_string_to_float(start)
    end_ratio = ratio_string_to_float(end)
    ratio = end_ratio / start_ratio 
    # print(f'calculated ratio is {ratio = }')
    if not find_closest: return (ratio)
    if .76 < ratio <= 1.6:
        return ratio
        # print(f'ratio in proper range {ratio = }')
    else:
        # print(f'out of range: {ratio = }')
        if ratio >= 1.5: 
            ratio = end_ratio / (start_ratio * 2)
            # print(f'new {ratio = }')
        elif ratio <= .75: 
            ratio = end_ratio * 2 / start_ratio
            # print(f'new {ratio = }')
    return (ratio)

print(f'distance from {ratio_strings[0,8]} to {ratio_strings[0,2]} is {Fraction(ratio_distance(ratio_strings[0,8], ratio_strings[0,2])).limit_denominator(100)}')
print(f'distance from {ratio_strings[5,0]} to {ratio_strings[12,0]} is {Fraction(ratio_distance(ratio_strings[5,0], ratio_strings[12,0])).limit_denominator(100)}')
ratio1 = all_ratio_strings[0]
ratio2 = all_ratio_strings[14]
# default behavior
print(f'distance from {ratio1} to {ratio2} is {ratio_distance(ratio1, ratio2, find_closest = False)} = {Fraction(ratio_distance(ratio1, ratio2)).limit_denominator(100)}')
print(f'distance from {ratio2} to {ratio1} is {ratio_distance(ratio2, ratio1, find_closest = False)} = {Fraction(ratio_distance(ratio2, ratio1)).limit_denominator(100)}')
# find_closest = False - just return the caluculated ratio without looking for a closer note by adjusting octaves of either note
print(f'distance from {ratio1} to {ratio2} is {ratio_distance(ratio1, ratio2, find_closest = False)} = {Fraction(ratio_distance(ratio1, ratio2, find_closest = False)).limit_denominator(100)}')
print(f'distance from {ratio2} to {ratio1} is {ratio_distance(ratio2, ratio1, find_closest = False)} = {Fraction(ratio_distance(ratio2, ratio1, find_closest = False)).limit_denominator(100)}')

distance from 3/2 to 9/8 is 3/2
distance from 32/21 to 8/7 is 3/2
distance from 1/1 to 15/8 is 1.875 = 15/16
distance from 15/8 to 1/1 is 0.5333333333333333 = 16/15
distance from 1/1 to 15/8 is 1.875 = 15/8
distance from 15/8 to 1/1 is 0.5333333333333333 = 8/15


In [75]:
# for each of the keys, build the chords from the scales created above.
# key is a string index into the keys dictionary: e.g. mode, ratio
# inversion is the string index into the inversions dictionary e.g. 'A_oton_1'
# the inversion includes the rank is which of the four tetrachords in each scale, A, B, C, or D
# And it specifies which note is on top

def build_chords(mode, ratio, rank, inversion):
    if (ratio in keys[mode]) and (inversion in inversions[rank][mode]): 
        chord = np.array([keys[mode][ratio][note] for note in (inversions[rank][mode][inversion])]) 
    else:
        print(f'in build_chords. Could not find {ratio = } with {mode} in {keys[mode] = } with {rank = }')
        chord = None
    return chord

In [76]:
# this function transform a note sequence dictionary into an array that can be passed to csound
# as I remember it also translates the duration and hold values into start time and duration 
def array_from_dict(time_step_dict):
    
    time_step_array = np.empty((len(time_step_dict), len(time_step_dict["instrument"])), dtype = float) # I may need all these to ge integer, but then create a t0 600 make it 10x faster
    # make sure you have the same number of values in each dictionary entry
    assert all([len(time_step_dict[time_step]) == len(time_step_dict["instrument"]) for time_step in time_step_dict.keys()]),\
        "not all items are same quantity"
    
    inx = 0
    for column in time_step_dict:
        time_step_array[inx] = time_step_dict[column]
        inx += 1
        
    
    return (time_step_array.T)

In [77]:
# This function takes a table number, glissando type, and ratio and returns an array that can be passed to csound to bend a note
def make_ftable_glissando(t_num, gliss_type, ratio):
    # to load a function table, we have to give csound some information about it in a preface:
    #                          +-- the function table number we are creating (passed to function)
    #                          |      +-- 0 start moving immediately
    #                          |      |  +-- the size of the array (prefers a power of two)
    #                          |      |  |    +-- the type of table format, in this case values that describe linear movement, negative means do not normalize
    #                          |      |  |    |     
    if gliss_type == 'slide':# |      |  |    |
        fn_preface = np.array([t_num, 0, 256, -7])
        # ;                  +-- start at 1:1, which is the current note
        # ;                  |  +-- take this amount of time at 1:1 16 out of 256 is 1/16 of the duration of the note
        # ;                  |  |   +-- stay at 1:1
        # ;                  |  |   |  +-- take this long to reach the next 
        # ;                  |  |   |  |    +-- target ratio, you want to go up or down this amount
        # ;                  |  |   |  |    |      +-- stay at 2nd note this long
        # ;                  |  |   |  |    |      |    + 
        #                    |  |   |  |    |      |    |
        fn_array = np.array([1, 16, 1, 128, ratio, 112, ratio])
    elif gliss_type == 'trill_2_step':
        fn_preface = np.array([t_num, 0, 256, -7])
        fn_array = np.array([1, 32, 1, 0, ratio, 32, ratio, 0, 1, 32, 1, 0, ratio, 160, ratio])
    elif gliss_type == 'trill_8_step':
        fn_preface = np.array([t_num, 0, 256, -7])
        fn_array = np.array([1, 16, 1, 0, ratio, 16, ratio, 0, 1, 16, 1, 0, ratio, 16, ratio, 0, 1, 16, 1, 0, ratio, 16, ratio, 0, 1, 16, 1, 0, ratio, 16, ratio, 
                             0, 1, 16, 1, 0, ratio, 16, ratio, 0, 1, 16, 1, 0, ratio, 16, ratio, 0, 1, 16, 1, 0, ratio, 16, ratio, 0, 1, 16, 1, 0, ratio, 16, ratio])
    else: print(f'invalid gliss type: {gliss_type}')
    fn_array_ready_to_load = np.concatenate((fn_preface,fn_array)) 
    return fn_array_ready_to_load

In [78]:
# take a scale as indexes and return the ratios as strings
def show_scale_ratios(scale):
    for note in scale:
        return(all_ratio_strings[scale])

In [79]:
def start_logger(fname = LOGNAME):
    if os.path.exists(LOGNAME):
            os.remove(LOGNAME) # make sure the log starts over with a fresh log file. Next line starts the logger.
    logger = logging.getLogger()
    fhandler = logging.FileHandler(filename=fname, mode='w')
    formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
    fhandler.setFormatter(formatter)
    logger.addHandler(fhandler)
    logger.setLevel(logging.DEBUG)

In [80]:
def flushMessages(cs, delay=0):
    s = ""
    if delay > 0:
        time.sleep(delay)
    for i in range(cs.messageCnt()):
        s += cs.firstMessage()
        cs.popFirstMessage()
    return s

def printMessages(cs, delay=0):
    s = flushMessages(cs, delay)
    this_many = 0
    if len(s)>0:
        logging.info(s)

In [81]:
# load the orchestra .csd file in to a long string that can be passed to csound
def load_csd(csd_file, strip_f0 = False):
    csd_content = ""
    lines = 0
    empty = False
    with open(csd_file,'r') as csd:
        while not empty:
            skip = False
            read_str = csd.readline()
            empty = not read_str
            lines += 1
            if (read_str.startswith('f0') or read_str.startswith('</CsScore' or read_str.startswith('</CsoundS'))) and strip_f0:
                skip = True
            if read_str.startswith('i') or read_str.startswith('t'):
                skip = True
                
            if not skip: csd_content += read_str
    csd.close()        
    return(csd_content, lines)


In [82]:
# start up an instance of csound so that you can pass commands to the started orchestra
def load_csound(csd_content):
    cs = ctcsound.Csound()    # create an instance of Csound
    cs.createMessageBuffer(0)    
    cs.setOption('-odac')
    cs.setOption("-G")  # Postscript output
    cs.setOption("-W")  # create a WAV format output soundfile
    printMessages(cs)
    cs.compileCsdText(csd_content)       # Compile Orchestra from String - already read in from a file 
    printMessages(cs)
    cs.start()             # When compiling from strings, this call is necessary before doing any performing
    flushMessages(cs)

    pt = ctcsound.CsoundPerformanceThread(cs.csound()) # Create a new CsoundPerformanceThread, passing in the Csound object
    pt.play()              # starts the thread, which is now running separately from the main thread. This 
                    # call is asynchronous and will immediately return back here to continue code
                    # execution.
    return (cs, pt)


In [83]:
# convert the durations into start_time["flute"] for when the note begins to play based on the sum of the prior durations.
def fix_start_duration_values(time_step_array, short_name):
    current_time = voice_time[short_name]["start_time"]
    temp_array = copy.deepcopy(time_step_array) # this is necessary because I mess with the start column in the time_step_array 
    inx = 0
    for row in temp_array:
        voice_time[short_name]["start_time"] += current_time
        current_time = row[1] # the 'start' column
        temp_array[inx,1] = voice_time[short_name]["start_time"]
        inx += 1
    return (temp_array)

In [84]:
# send in a starting table number, and two arrays of note indeciiss, and it will return an ftable that can be passed to csound with a set of slides for each note to reach the second set of notes
# for example, start_table = 800, chord_1 = np.array([36, 40, 44, 32]), chord_2 = np.array([34, 38, 42, 46]), slide='slide'
# will return four slide arrays to pass to csound making ftables 800, 801, 802, & 803
def build_slides(start_table, chord_1, chord_2, gliss_type = 'slide'):
    if gliss_type in (['slide', 'trill_2_step', 'trill_8_step']):
        gliss_f_table = np.array([make_ftable_glissando(start_table + i, gliss_type, ratio_distance(all_ratio_strings[a],all_ratio_strings[b])) for i, (a, b) in enumerate(zip(chord_1, chord_2))])
        
    else: 
        print(f'invalid gliss type: {gliss_type = }')
        gliss_f_table = 1.0
    return gliss_f_table

In [227]:
# this is a function call to build an octave boost mask that can be applied to keep scales always moving up.\n",
# scale is an array of (note,) 
# return a mask that indicates by a 1 that the note should be increased an octave, or a zero that it should not\n",
def build_scale_mask(scale):
    boost_octave = 1
    no_boost = 0
    mask = np.zeros(scale.shape,dtype=int) 
    prev_note = ratio_string_to_float(all_ratio_strings[scale[0]]) # this assumes shape in (note,) dangerous
    n = 0
    for note in scale:
        note_ratio = ratio_string_to_float(all_ratio_strings[note])
        if note_ratio < prev_note:
            mask[n] = boost_octave
        else: mask[n] = no_boost
        n += 1
    return (mask)

## Explore the Dictionaries and Functions we have created
The next cells are optional to show what we have created so far. It shows how the dictionaries that describe scales can be used to generate sequences of notes. The cells produce examples of the available scales and chords. To see all the possible combinations, set variable first_item_only to False. To limit the output, set it to True.

In [85]:
# List the thirds in ranks E, F, G, H
print(f'{ratio_string_to_float("5/4") = }')
print(f'{ratio_string_to_float("21/17") = }')
print(f'{ratio_string_to_float("11/9") = }')
print(f'{ratio_string_to_float("6/5") = }')
print(f'{ratio_string_to_float("7/6") = }')
print(f'{ratio_string_to_float("8/7") = }')

ratio_string_to_float("5/4") = 1.25
ratio_string_to_float("21/17") = 1.2352941176470589
ratio_string_to_float("11/9") = 1.2222222222222223
ratio_string_to_float("6/5") = 1.2
ratio_string_to_float("7/6") = 1.1666666666666667
ratio_string_to_float("8/7") = 1.1428571428571428


In [86]:
# Here is what some scales looks like. Set first_item_only to False to show them all
# It consists of indeciis into the f3 function table, from which csound returns the octave fraction and then the Hertz of the note.
# this cell demonstrates how to calculate a mask that can be applied to a scale to ensure it always goes up regardless of the octave

for rank in scales.keys(): # A, B, C, D, E, F, G, H
    mode = 'oton'
    for root in ratio_strings[:,0]:
        scale_index = build_scales(mode, root, rank)
        print(f'{rank}, {mode}, {root}, '
            f'ratios in scale_index: {[all_ratio_strings[step] for step in scale_index]} '      
            f'floating point ratios in scale_index: {[round(ratio_string_to_float(all_ratio_strings[step]),2) for step in scale_index]} '
            f'mask: {build_scale_mask(scale_index)}')                      
        if first_item_only: break
    mode = 'uton'
    for root in ratio_strings[0,:]:     
        scale_index = build_scales(mode, root, rank) # 
        print(f'{rank}, {mode}, {root}, '
            f'ratios in my_scale: {[all_ratio_strings[step] for step in scale_index]} '      
            f'floating point ratios in my_scale: {[round(ratio_string_to_float(all_ratio_strings[step]),2) for step in scale_index]} '
            f'mask: {build_scale_mask(scale_index)}')    
        if first_item_only: break

A, oton, 1/1, ratios in scale_index: ['1/1', '9/8', '5/4', '11/8', '3/2', '13/8', '7/4', '15/8'] floating point ratios in scale_index: [1.0, 1.12, 1.25, 1.38, 1.5, 1.62, 1.75, 1.88] mask: [0 0 0 0 0 0 0 0]
A, uton, 1/1, ratios in my_scale: ['4/3', '16/11', '8/5', '16/9', '1/1', '16/15', '8/7', '16/13'] floating point ratios in my_scale: [1.33, 1.45, 1.6, 1.78, 1.0, 1.07, 1.14, 1.23] mask: [0 0 0 0 1 1 1 1]
B, oton, 1/1, ratios in scale_index: ['9/8', '5/4', '11/8', '3/2', '13/8', '7/4', '15/8', '1/1'] floating point ratios in scale_index: [1.12, 1.25, 1.38, 1.5, 1.62, 1.75, 1.88, 1.0] mask: [0 0 0 0 0 0 0 1]
B, uton, 1/1, ratios in my_scale: ['16/15', '8/7', '16/13', '4/3', '16/11', '8/5', '16/9', '1/1'] floating point ratios in my_scale: [1.07, 1.14, 1.23, 1.33, 1.45, 1.6, 1.78, 1.0] mask: [0 0 0 0 0 0 0 1]
C, oton, 1/1, ratios in scale_index: ['17/16', '19/16', '21/16', '23/16', '25/16', '27/16', '29/16', '31/16'] floating point ratios in scale_index: [1.06, 1.19, 1.31, 1.44, 1.56, 1

In [87]:
# print(f'{[build_chords("oton", "16/9","A", chord_inversion) for chord_inversion in inversions["A"]["oton"]]}')
# the resulting notes are the scale degree in the 256 notes in the tonality diamond and can be sent to csound to create that note or chord
# or if you just want to know the ratios, use the results to index the ratio_strings array.
# inspect an interesting chord in different inversions
for inv in inversions["E"]["uton"]:
    print(f'{build_chords("uton", "1/1","E", chord_inversion)}')
    print(f'{[all_ratio_strings[note] for note in build_chords("uton", "1/1","E", inv)]}')

[128  64   0 208]
['4/3', '8/5', '1/1', '32/29']
[128  64   0 208]
['8/5', '1/1', '32/29', '4/3']
[128  64   0 208]
['1/1', '32/29', '4/3', '8/5']
[128  64   0 208]
['4/3', '8/5', '1/1', '32/29']


In [88]:
# show me all the ranks of the otonal 8 note scales with a root of 1:1
ratio = '1/1'
mode = 'uton'
print(f'{[("rank:", rank, "indeciis: ", build_scales(mode, ratio, rank), "ratios: ", show_scale_ratios(build_scales(mode, ratio, rank))) for rank in scales.keys()]}')

[('rank:', 'A', 'indeciis: ', array([128,  96,  64,  32,   0, 224, 192, 160]), 'ratios: ', array(['4/3', '16/11', '8/5', '16/9', '1/1', '16/15', '8/7', '16/13'],
      dtype='<U5')), ('rank:', 'B', 'indeciis: ', array([224, 192, 160, 128,  96,  64,  32,   0]), 'ratios: ', array(['16/15', '8/7', '16/13', '4/3', '16/11', '8/5', '16/9', '1/1'],
      dtype='<U5')), ('rank:', 'C', 'indeciis: ', array([208, 176, 144, 112,  80,  48,  16, 240]), 'ratios: ', array(['32/29', '32/27', '32/25', '32/23', '32/21', '32/19', '32/17',
       '32/31'], dtype='<U5')), ('rank:', 'D', 'indeciis: ', array([240, 208, 176, 144, 112,  80,  48,  16]), 'ratios: ', array(['32/31', '32/29', '32/27', '32/25', '32/23', '32/21', '32/19',
       '32/17'], dtype='<U5')), ('rank:', 'E', 'indeciis: ', array([128,  96,  64,  32,   0, 240, 208, 176]), 'ratios: ', array(['4/3', '16/11', '8/5', '16/9', '1/1', '32/31', '32/29', '32/27'],
      dtype='<U5')), ('rank:', 'F', 'indeciis: ', array([  0, 224, 192, 160, 128,  96,  

In [89]:
# to find the distance bewteen two ratios, divide the larger ratio by the smaller ratio.
# To divide fractions, multiply the larger ratio times the reciprocal of the smaller ratio
print(f'{Fraction((32/17) / (32/21)).limit_denominator(100)}') # divide the larger by the smaller ratio
print(f'{Fraction((16/17) * (8/7)).limit_denominator(100)}') # the equivalent calculation is to multiply by the reciprocal of the 2nd number 

21/17
71/66


In [90]:
# I am looking for a certain ratio (3/2 or 3/4, 2/3 or 4/3) which is required for a decent scale.
print('oton intervals with a perfect 3:2')                
for prev in np.arange(0,16,1):
    for item in np.arange(0,16,1):
        if prev != item:
            save_ratio = ratio_distance(ratio_strings[0,prev], ratio_strings[0,item])
            # if 0.75 < save_ratio < 1.5:
            if save_ratio in ([3/2, 3/4]): # perfect fifth for overtone scales
                print(f'({prev}, {item}): {Fraction(save_ratio).limit_denominator(max_denominator = 100)}',end=', ')
print('\nuton scales with a perfect 4:3')                
# Otonal are inverted:
for prev in np.arange(0,16,1):
    for item in np.arange(0,16,1):
        if prev != item:
            save_ratio = ratio_distance(ratio_strings[0,prev], ratio_strings[0,item])
            # if 0.75 < save_ratio < 1.5:
            if save_ratio in ([4/3, 2/3]): # perfect 4th for undertone scales. These transform into perfect 5th when the scale is pulled
                print(f'({prev}, {item}): {Fraction(save_ratio).limit_denominator(max_denominator = 100)}',end=', ')

oton intervals with a perfect 3:2
(0, 8): 3/2, (2, 11): 3/2, (4, 14): 3/2, (8, 2): 3/4, (12, 5): 3/4, 
uton scales with a perfect 4:3
(2, 8): 4/3, (5, 12): 4/3, (8, 0): 2/3, (11, 2): 2/3, (14, 4): 2/3, 

In [92]:
# build_scales("oton", "16/9", rank) - these are 8 note scales, based on the mode, root, and rank
# build_scales(mode, ratio, rank):
# first_item_only = True
for mode in keys:
    for root in keys[mode]:
        print(f'16 note scale in keys[{mode}, {root}], {keys[mode][root]}')
        for rank in scales:
            print(f'{rank}: {build_scales(mode, root, rank)}')
            if rank == 'B' and first_item_only: break
        if root == '1/1' and first_item_only: break

16 note scale in keys[oton, 1/1], [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]
A: [ 0  2  4  6  8 10 12 14]
B: [ 2  4  6  8 10 12 14  0]
16 note scale in keys[uton, 1/1], [  0  16  32  48  64  80  96 112 128 144 160 176 192 208 224 240]
A: [128  96  64  32   0 224 192 160]
B: [224 192 160 128  96  64  32   0]


In [225]:
# confirm that all scales in all used keys, all ranks are all going from low to high, and distance from 0 to 7 less than OCTAVE_SIZE
# how many scales? 8 ranks, two modes for each rank, each mode has 16 scales. 8 * 2 * 16 = 256 scales of 8 notes each. 
lowest_travel = 255
highest_travel = 0
# iterate on scale_mask[rank, mode, key] 
# first_item_only = False
first_item_only = True
many_scales = 0
for rank in scales: # 'oton' and 'uton'
    print(f'{rank = }')
    for mode in scales[rank]: # 'A', 'B', etc.
        print(f'{mode = }')
        for ratio in keys[mode]: # 1/1, 16/9, etc
            # print(f'{ratio = }')
            built_scale = build_scales(mode, ratio, rank)
            built_mask = build_scale_mask(built_scale)
            print(f'{rank}, {mode}, {ratio}:\t'
                f'{[round(ratio_string_to_float(ratio),2) for ratio in show_scale_ratios(built_scale)]}\t {built_mask}')
            many_scales += 1
    if first_item_only: break
print(f'how {many_scales = }')

rank = 'A'
mode = 'oton'
A, oton, 1/1:	[1.0, 1.12, 1.25, 1.38, 1.5, 1.62, 1.75, 1.88]	 [0 0 0 0 0 0 0 0]
A, oton, 32/17:	[1.88, 1.06, 1.18, 1.29, 1.41, 1.53, 1.65, 1.76]	 [0 1 1 1 1 1 1 1]
A, oton, 16/9:	[1.78, 1.0, 1.11, 1.22, 1.33, 1.44, 1.56, 1.67]	 [0 1 1 1 1 1 1 1]
A, oton, 32/19:	[1.68, 1.89, 1.05, 1.16, 1.26, 1.37, 1.47, 1.58]	 [0 0 1 1 1 1 1 1]
A, oton, 8/5:	[1.6, 1.8, 1.0, 1.1, 1.2, 1.3, 1.4, 1.5]	 [0 0 1 1 1 1 1 1]
A, oton, 32/21:	[1.52, 1.71, 1.9, 1.05, 1.14, 1.24, 1.33, 1.43]	 [0 0 0 1 1 1 1 1]
A, oton, 16/11:	[1.45, 1.64, 1.82, 1.0, 1.09, 1.18, 1.27, 1.36]	 [0 0 0 1 1 1 1 1]
A, oton, 32/23:	[1.39, 1.57, 1.74, 1.91, 1.04, 1.13, 1.22, 1.3]	 [0 0 0 0 1 1 1 1]
A, oton, 4/3:	[1.33, 1.5, 1.67, 1.83, 1.0, 1.08, 1.17, 1.25]	 [0 0 0 0 1 1 1 1]
A, oton, 32/25:	[1.28, 1.44, 1.6, 1.76, 1.92, 1.04, 1.12, 1.2]	 [0 0 0 0 0 1 1 1]
A, oton, 16/13:	[1.23, 1.38, 1.54, 1.69, 1.85, 1.0, 1.08, 1.15]	 [0 0 0 0 0 1 1 1]
A, oton, 32/27:	[1.19, 1.33, 1.48, 1.63, 1.78, 1.93, 1.04, 1.11]	 [0 0 0 0 0 

In [94]:
# confirm that you have created chords properly
# build_chords(mode, ratio, rank, inversion):
# build_chords(mode, ratio, rank, inversion):
oct = 1
for rank in inversions: # rank A, B, C, D
    # print(f'{rank = }')
    for mode in keys: # 'oton', 'uton'
        # print(f'{mode = }')
        for ratio in keys[mode]: # '1/1', '32/17', '16/9', etc
            # print(f'{ratio = }')
            for inv in inversions[rank][mode]: # 1, 2, 3, 4
                # print(f'{inv = }')
                print(f'{rank = }, {mode = }, {ratio = }, {inv = }, : {show_scale_ratios(build_chords(mode, ratio, rank, inv))}')
                # if first_item_only: break
            if first_item_only: break
    if first_item_only: break

rank = 'A', mode = 'oton', ratio = '1/1', inv = 1, : ['1/1' '5/4' '3/2' '7/4']
rank = 'A', mode = 'oton', ratio = '1/1', inv = 2, : ['5/4' '3/2' '7/4' '1/1']
rank = 'A', mode = 'oton', ratio = '1/1', inv = 3, : ['3/2' '7/4' '1/1' '5/4']
rank = 'A', mode = 'oton', ratio = '1/1', inv = 4, : ['7/4' '1/1' '5/4' '3/2']
rank = 'A', mode = 'uton', ratio = '1/1', inv = 1, : ['8/7' '4/3' '8/5' '1/1']
rank = 'A', mode = 'uton', ratio = '1/1', inv = 2, : ['1/1' '8/7' '4/3' '8/5']
rank = 'A', mode = 'uton', ratio = '1/1', inv = 3, : ['8/5' '1/1' '8/7' '4/3']
rank = 'A', mode = 'uton', ratio = '1/1', inv = 4, : ['4/3' '8/5' '1/1' '8/7']


In [95]:
# pick 3 random note orders
for i in np.arange(3):
    print(f'{rng.choice(([1,2,3,4,5,6,7,8]), size = 8, replace = False, shuffle = True)}')

[6 8 1 2 4 5 7 3]
[7 2 4 6 3 5 1 8]
[2 7 6 5 4 1 3 8]


## Play some sounds on Csound
The next cells assembles a pair of chords and slides from one to the next, and back again. It calls the build_chords function, passing in the root ratio, the mode, and the rank for each of the two chords. It then builds a slide or trill between the two chords, such that each note of the first chord moves to the corresponding note of the next chord and back again. It then starts up an instance of csound, loads a skeleton csd file to make music with sample files.

I then build a note dictionary with all the data elements required to make a note in the csound orchestra created in the skelton csd file. It does some tranformation of the dictionary into a numpy array, and modifies two fields of the array to fix the start times and duration values. Csound needs start time and durations, but I prefer thinking in terms of duration as the time between one note and the next, and prefer a separate value, I call 'hold', for how long the note should be held. This simplifies creating chords that sound at the same time and last a certain amount of time before the next note is played. Each instrument has a separate value for start time. In this way the instrument can play multiple notes at the same time, and moves the time forward based on the duration values. 

For example, suppose you want to play two separate voices, one for the flute and another for the bassoon. Each should play notes at the beginning of the piece. We can specify a set of durations, which is how long until the next note in this instrument is played (duration value) and the amount of time each note should be held (hold value). In the first case below, the hold value is set to 5, 5, 5, 5, but the duration is set to 0, 0, 0, 5. This creates a chord played across four flutes, held for 5 seconds. If the next hold value is also set to 5, 5, 5, 5 and the duration set to 0, 0, 0, 5, then the next chord is played for 5 seconds across four flutes. At the same time, we can have another note dictionary that plays a separate set of scales on the bassoon. Oh, and the chords include a slide from one chord to the next, and the next chord slides back to where it started. 

In [190]:
# create a sliding chord from one to another and back
oct = 4
ratio = '1/1'
mode = 'oton'
# first chord:
rank_1 = 'A'
inv_1 = 2
# second chord:
rank_2 = 'B'
inv_2 = 1
gliss_type = 'slide'
# make two chords on these values
chord_1 = build_chords(mode, ratio, rank_1, inv_1)
chord_2 = build_chords(mode, ratio, rank_2, inv_2)
# make slides to move from the first chord to the second (down) and back (up)
gliss_down = build_slides(801, chord_1, chord_2, gliss_type = gliss_type)
gliss_up = build_slides(805, chord_2, chord_1, gliss_type = gliss_type)
# build the scales to be played on the bassoon
scale_1 = build_scales(mode, ratio, rank_1)
scale_2 = build_scales(mode, ratio, rank_2)
mask_1 = build_scale_mask(scale_1) # which notes must be moved up an octave so that the scale goes from down to up
mask_2 = build_scale_mask(scale_2) # same for the second scale
print(f'{scale_1 = }, {mask_1 = }')
print(f'{scale_2 = }, {mask_2 = }')
print(f'gliss down values for table number and target gliss: {[(np.round(column[0], 3), np.round(column[8], 3)) for column in gliss_down]}')
print(f'gliss up values for table number and target gliss: {[(np.round(column[0], 3), np.round(column[8], 3)) for column in gliss_up]}')

scale_1 = array([ 0,  2,  4,  6,  8, 10, 12, 14]), mask_1 = array([0, 0, 0, 0, 0, 0, 0, 0])
scale_2 = array([ 2,  4,  6,  8, 10, 12, 14,  0]), mask_2 = array([0, 0, 0, 0, 0, 0, 0, 1])
gliss down values for table number and target gliss: [(801.0, 0.9), (802.0, 0.917), (803.0, 0.929), (804.0, 0.938)]
gliss up values for table number and target gliss: [(805.0, 1.111), (806.0, 1.091), (807.0, 1.077), (808.0, 1.067)]


In [191]:
# the last slide is 1.875, which is from 14 (15:8) to 0 (1:1). I wanted 15:8 to 2:1.
# how can I change the function build_slides to make that check and choose the nearest alternative?
print(f'{scale_1[-1] = }, {all_ratio_strings[scale_1[-1]] = }')
print(f'{scale_2[-1] = }, {all_ratio_strings[scale_2[-1]] = }')

scale_1[-1] = 14, all_ratio_strings[scale_1[-1]] = '15/8'
scale_2[-1] = 0, all_ratio_strings[scale_2[-1]] = '1/1'


In [201]:
# 
#
# Start up an instance of csound
#
#
if not csound_running: 
    start_logger()
    csd_content, lines = load_csd(CSD_FILE)
    logging.info(f'Starting to log to {LOGNAME}')
    logging.info(f'Loaded the csd file {CSD_FILE}. There are {lines} lines read containg {len(csd_content)} bytes')
    cs, pt = load_csound(csd_content)
    csound_running = True
else: print(f'csound is already running')

0dBFS level = 32768.0
--Csound version 6.16 (double samples) Aug 10 2021
[commit: none]
libsndfile-1.0.31
end of score.		   overall amps:      0.0
	   overall samples out of range:        0
0 errors in performance
Elapsed time at end of performance: real: 9.858s, CPU: 0.028s


In [202]:
notes = 8
scale_notes = scale_1.shape[0] + scale_2.shape[0]
scale_dur = 10 / scale_notes # count how many notes in the scales and divide 10 seconds by that amount for each duration
chord_duration = np.array([0, 0, 0, 5.2, 0, 0, 0, 5.2], dtype = float) # I need to keep this float to support faster passages.
scale_duration = np.ones((scale_notes,), dtype = int) * scale_dur 
hold = 5
velocity = 60
upsamples = np.array([1,1])
volume = 30
gliss1 = 0
gliss2 = 0
gliss3 = 0
envelope = 1
oct = 3
bfl = 'bfl' # finger piano short name
bss = 'bss'

time_step_dict = {bfl: {
                  'instrument': np.ones((notes,), dtype = int),
                  'duration': chord_duration,
                  'hold': np.ones((notes,), dtype = int) * hold,
                  'velocity': np.ones((notes,), dtype = int) * velocity,
                  'tone': np.concatenate((chord_1, chord_2)),
                  'octave': np.ones((notes,), dtype = int) * oct,
                  'voice': np.repeat(voice_time[bfl]["number"],notes),
                  'stereo': np.ones((notes,), dtype = int) * notes, 
                  'l_envelope': np.ones((notes,), dtype = int) * envelope, 
                  'gliss1': np.concatenate((gliss_down[:,0], gliss_up[:,0])),
                  'upsample': np.concatenate((np.repeat(upsamples[0],4), np.repeat(upsamples[1],4))),
                  'r_envelope': np.ones((notes,), dtype = int) * envelope, 
                  'gliss2' : np.ones((notes,), dtype = int) * gliss2,
                  'gliss3': np.ones((notes,), dtype = int) * gliss2,
                  'volume': np.ones((notes,), dtype = int) * volume},
            bss: {
                  'instrument': np.ones((scale_notes,), dtype = int),
                  'duration': scale_duration,
                  'hold': np.ones((scale_notes,), dtype = int) * scale_dur + .1,
                  'velocity': np.ones((scale_notes,), dtype = int) * velocity,
                  'tone': np.concatenate((scale_1, scale_2)),
                  'octave': np.concatenate((mask_1, mask_2)) + oct,
                  'voice': np.repeat(voice_time[bss]["number"], scale_notes),
                  'stereo': np.ones((scale_notes,), dtype = int) * notes, 
                  'l_envelope': np.ones((scale_notes,), dtype = int) * 2, 
                  'gliss1': np.ones((scale_notes,), dtype = int) * gliss2, 
                  'upsample': np.ones((scale_notes,), dtype = int) * 1,
                  'r_envelope': np.ones((scale_notes,), dtype = int) * 2, 
                  'gliss2' : np.ones((scale_notes,), dtype = int) * gliss2,
                  'gliss3': np.ones((scale_notes,), dtype = int) * gliss2,
                  'volume': np.ones((scale_notes,), dtype = int) * volume}
                 }
                 
                
pp.pprint(time_step_dict)

{'bfl': {'instrument': array([1, 1, 1, 1, 1, 1, 1, 1]),
         'duration': array([0. , 0. , 0. , 5.2, 0. , 0. , 0. , 5.2]),
         'hold': array([5, 5, 5, 5, 5, 5, 5, 5]),
         'velocity': array([60, 60, 60, 60, 60, 60, 60, 60]),
         'tone': array([ 4,  8, 12,  0,  2,  6, 10, 14]),
         'octave': array([3, 3, 3, 3, 3, 3, 3, 3]),
         'voice': array([6, 6, 6, 6, 6, 6, 6, 6]),
         'stereo': array([8, 8, 8, 8, 8, 8, 8, 8]),
         'l_envelope': array([1, 1, 1, 1, 1, 1, 1, 1]),
         'gliss1': array([801., 802., 803., 804., 805., 806., 807., 808.]),
         'upsample': array([1, 1, 1, 1, 1, 1, 1, 1]),
         'r_envelope': array([1, 1, 1, 1, 1, 1, 1, 1]),
         'gliss2': array([0, 0, 0, 0, 0, 0, 0, 0]),
         'gliss3': array([0, 0, 0, 0, 0, 0, 0, 0]),
         'volume': array([30, 30, 30, 30, 30, 30, 30, 30])},
 'bss': {'instrument': array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
         'duration': array([0.625, 0.625, 0.625, 0.625, 0.625,

In [203]:
# make the bass flute part
print(f'{[len(time_step_dict[bss][a]) for a in time_step_dict[bss].keys()] = }')

[len(time_step_dict[bss][a]) for a in time_step_dict[bss].keys()] = [16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16]


In [204]:
voice_time[bfl]["start_time"] = 0

In [205]:
time_step_flute = fix_start_duration_values(array_from_dict(time_step_dict[bfl]), voice_time, bfl)   

In [206]:
print(f'{time_step_flute = },\n{voice_time[bfl]["start_time"] = }')

time_step_flute = array([[  1. ,   0. ,   5. ,  60. ,   4. ,   3. ,   6. ,   8. ,   1. ,
        801. ,   1. ,   1. ,   0. ,   0. ,  30. ],
       [  1. ,   0. ,   5. ,  60. ,   8. ,   3. ,   6. ,   8. ,   1. ,
        802. ,   1. ,   1. ,   0. ,   0. ,  30. ],
       [  1. ,   0. ,   5. ,  60. ,  12. ,   3. ,   6. ,   8. ,   1. ,
        803. ,   1. ,   1. ,   0. ,   0. ,  30. ],
       [  1. ,   0. ,   5. ,  60. ,   0. ,   3. ,   6. ,   8. ,   1. ,
        804. ,   1. ,   1. ,   0. ,   0. ,  30. ],
       [  1. ,   5.2,   5. ,  60. ,   2. ,   3. ,   6. ,   8. ,   1. ,
        805. ,   1. ,   1. ,   0. ,   0. ,  30. ],
       [  1. ,   5.2,   5. ,  60. ,   6. ,   3. ,   6. ,   8. ,   1. ,
        806. ,   1. ,   1. ,   0. ,   0. ,  30. ],
       [  1. ,   5.2,   5. ,  60. ,  10. ,   3. ,   6. ,   8. ,   1. ,
        807. ,   1. ,   1. ,   0. ,   0. ,  30. ],
       [  1. ,   5.2,   5. ,  60. ,  14. ,   3. ,   6. ,   8. ,   1. ,
        808. ,   1. ,   1. ,   0. ,   0. ,  30. ]]),
voic

In [207]:
# make the basssoon part
voice_time[bss]["start_time"] = 0
time_step_bassoon = fix_start_duration_values(array_from_dict(time_step_dict[bss]), voice_time, bss)   
print(f'{time_step_bassoon = },\n{voice_time[bss]["start_time"] = }')

time_step_bassoon = array([[ 1.   ,  0.   ,  0.725, 60.   ,  0.   ,  3.   ,  9.   ,  8.   ,
         2.   ,  0.   ,  1.   ,  2.   ,  0.   ,  0.   , 30.   ],
       [ 1.   ,  0.625,  0.725, 60.   ,  2.   ,  3.   ,  9.   ,  8.   ,
         2.   ,  0.   ,  1.   ,  2.   ,  0.   ,  0.   , 30.   ],
       [ 1.   ,  1.25 ,  0.725, 60.   ,  4.   ,  3.   ,  9.   ,  8.   ,
         2.   ,  0.   ,  1.   ,  2.   ,  0.   ,  0.   , 30.   ],
       [ 1.   ,  1.875,  0.725, 60.   ,  6.   ,  3.   ,  9.   ,  8.   ,
         2.   ,  0.   ,  1.   ,  2.   ,  0.   ,  0.   , 30.   ],
       [ 1.   ,  2.5  ,  0.725, 60.   ,  8.   ,  3.   ,  9.   ,  8.   ,
         2.   ,  0.   ,  1.   ,  2.   ,  0.   ,  0.   , 30.   ],
       [ 1.   ,  3.125,  0.725, 60.   , 10.   ,  3.   ,  9.   ,  8.   ,
         2.   ,  0.   ,  1.   ,  2.   ,  0.   ,  0.   , 30.   ],
       [ 1.   ,  3.75 ,  0.725, 60.   , 12.   ,  3.   ,  9.   ,  8.   ,
         2.   ,  0.   ,  1.   ,  2.   ,  0.   ,  0.   , 30.   ],
       [ 1.   ,  4.37

In [208]:
for row in gliss_down: # pass some f tables for glissandi to the instance of csound
    pt.scoreEvent(0, 'f', row)
    print(f'{row[8] = }')
                  
for row in gliss_up: # pass some f tables for glissandi to the instance of csound
    pt.scoreEvent(0, 'f', row)
    print(f'{row[8] = }')
                  
for row in time_step_flute: # send the note information to csound
    pt.scoreEvent(0, 'i', row)
    print(f'[{[row[item] for item in ([1, 4, 9])]}') # print some of the values, start time, note index, glide index

for row in time_step_bassoon: # send the note information to csound
    pt.scoreEvent(0, 'i', row)
    print(f'[{[row[item] for item in ([1, 4, 9])]}') # print some of the values, start time, note index, glide index    

row[8] = 0.9
row[8] = 0.9166666666666666
row[8] = 0.9285714285714286
row[8] = 0.9375
row[8] = 1.1111111111111112
row[8] = 1.0909090909090908
row[8] = 1.0769230769230769
row[8] = 1.0666666666666667
[[0.0, 4.0, 801.0]
[[0.0, 8.0, 802.0]
[[0.0, 12.0, 803.0]
[[0.0, 0.0, 804.0]
[[5.2, 2.0, 805.0]
[[5.2, 6.0, 806.0]
[[5.2, 10.0, 807.0]
[[5.2, 14.0, 808.0]
[[0.0, 0.0, 0.0]
[[0.625, 2.0, 0.0]
[[1.25, 4.0, 0.0]
[[1.875, 6.0, 0.0]
[[2.5, 8.0, 0.0]
[[3.125, 10.0, 0.0]
[[3.75, 12.0, 0.0]
[[4.375, 14.0, 0.0]
[[5.0, 2.0, 0.0]
[[5.625, 4.0, 0.0]
[[6.25, 6.0, 0.0]
[[6.875, 8.0, 0.0]
[[7.5, 10.0, 0.0]
[[8.125, 12.0, 0.0]
[[8.75, 14.0, 0.0]
[[9.375, 0.0, 0.0]


In [200]:
#
#
# stop the csound instance
#
#
if csound_running:
    printMessages(cs)
    pt.stop() # this is important I think. It closes the output file.
    pt.join()   
    printMessages(cs)    
    cs.reset()
    csound_running = False
else:
    print(f"csound isn't running")

0dBFS level = 32768.0
--Csound version 6.16 (double samples) Aug 10 2021
[commit: none]
libsndfile-1.0.31


In [297]:
print(f'{[len(time_step_dict[time_step]) == len(time_step_dict["instrument"]) for time_step in time_step_dict.keys()]}')
print(f'{time_step_dict.keys() = }')
print(f'{[time_step_dict[time_step] for time_step in time_step_dict.keys()]}')
print(f'{len(time_step_dict) = }')

[True, True, True, True, True, True, True, True, True, True, True, True, True, True, True]
time_step_dict.keys() = dict_keys(['instrument', 'duration', 'hold', 'velocity', 'tone', 'octave', 'voice', 'stereo', 'l_envelope', 'gliss1', 'upsample', 'r_envelope', 'gliss2', 'gliss3', 'volume'])
[array([1, 1, 1, 1, 1, 1, 1, 1]), array([0. , 0. , 0. , 5.2, 0. , 0. , 0. , 5.2]), array([5, 5, 5, 5, 5, 5, 5, 5]), array([60, 60, 60, 60, 60, 60, 60, 60]), array([ 6, 10, 14,  2,  1,  5,  9, 13]), array([3, 3, 3, 3, 3, 3, 3, 3]), array([6, 6, 6, 6, 6, 6, 6, 6]), array([8, 8, 8, 8, 8, 8, 8, 8]), array([1, 1, 1, 1, 1, 1, 1, 1]), array([800., 801., 802., 803., 804., 805., 806., 807.]), array([  1,   1,   1,   1, 255, 255, 255, 255]), array([1, 1, 1, 1, 1, 1, 1, 1]), array([0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0]), array([30, 30, 30, 30, 30, 30, 30, 30])]
len(time_step_dict) = 15


In [ ]:
# ['instrument', 'start', 'duration', 'hold', 'velocity', 'tone', 'octave', 'voice', 'stereo', 
# 'l_envelope', 'gliss1', 'upsample', 'r_envelope', 'gliss2', 'gliss3', 'volume', 'elapsed']
print(f'{voice_time["flute"]["voice_number"] = }')

temp_array = copy.deepcopy(time_step_array) # this is necessary because I mess with the start column in the time_step_array 

#       +-- f says this is a function 
#       | +-- function is called # 330
#       | |   +-- start at time 0
#       | |   | +-- table has 256 time steps
#       | |   | |   +--  means don't rescale, 7 means straight lines
#       | |   | |   |  +--  table value starts at 1, meaning no change to pitch at the start
#       | |   | |   |  | +-- take this much time to reach next table value
#       | |   | |   |  | |  +-- still at no change to pitch
#       | |   | |   |  | |  | +-- over the next 128 time steps gradually go down towards 0.9375
#       | |   | |   |  | |  | |   +-- reach this value
#       | |   | |   |  | |  | |   |       +-- stay here for this number of time steps
#       | |   | |   |  | |  | |   |       |   +-- end here
#       | |   | |   |  | |  | |   |       |   |          +-- this is the number to send to csound to select ftable number 330. 29 + 301 = 330
f330 = (330, 0, 256, -7, 1, 16, 1, 128, 0.93750, 112, 0.93750) # 29
f332 = (332, 0, 256, -7, 1, 16, 1, 128, 0.92857, 112, 0.92857) # 31 
f334 = (334, 0, 256, -7, 1, 16, 1, 128, 0.91667, 112, 0.91667) # 33 
f336 = (336, 0, 256, -7, 1, 16, 1, 128, 0.90000, 112, 0.90000) # 35 
# mess with these

# f330 = (330, 0, 256, -7, 1, 16, 1, 128, 0.093750, 112, 0.093750) # 29
# f332 = (332, 0, 256, -7, 1, 16, 1, 128, 0.092857, 112, 0.092857) # 31 
# f334 = (334, 0, 256, -7, 1, 16, 1, 128, 0.091667, 112, 0.091667) # 33 
# f336 = (336, 0, 256, -7, 1, 16, 1, 128, 0.090000, 112, 0.090000) # 35 


pt.scoreEvent(0, 'f', f330)
pt.scoreEvent(0, 'f', f332)
pt.scoreEvent(0, 'f', f334)
pt.scoreEvent(0, 'f', f336)

voice_time[short_name]["start_time"] = 0 # convert the durations into start_time["flute"] for when the note begins to play based on the sum of the prior durations.
# print (f'{temp_array.shape = }')
current_duration = 0
inx = 0
for row in temp_array:
    voice_time[short_name]["start_time"] += current_duration
    current_duration = row[1] # the 'start' column
    # print(f'row number: {inx = }, before {current_duration = }, {voice["flute"]["start_time"] = }', end = '\t')
    temp_array[inx,1] = voice_time[short_name]["start_time"]
    # print(f'after {row[1] = }')
    pt.scoreEvent(0, 'i', row)
    inx += 1
print(f'{voice_time[short_name]["start_time"] = }')    
print(f'inst\tstart\thold\tvel\tton\toct\tvoi\tste\tEnv\tGli\tups\tREn\t2gl\t3gj\tvol')
print(f'0\t1\t2\t3\t4\t5\t6\t7\t8\t9\t10\t11\t12\t13\t14')
print(f'------- ------- ------- ------- ------- ------- ------- ------- ------- ------- ------- ------- ------- ------- -------')
for row in temp_array:
    for field in row:
        print(f'{round(field,2)}',end='\t')
    print()

In [ ]:
p.printMessages(cs)
# delay_time =  max(csound_params["min_delay"],len(pfields) // 50) # need enough time to prevent csound being told to stop 
# print(f'about to delay to allow ctcsound to process the notes. delay_time: {delay_time}')
last_start = pfields[-1][1]
print(f'last start time was at {round(last_start,1)}. Set f0 to {round(last_start+1,1)}')
# time.sleep(delay_time) # once you hit the next line csound stops
pt.stop() # this is important I think. It closes the output file.
pt.join()   
p.printMessages(cs)    
cs.reset()

## Size of the Tonality Diamond, where n is the limit:
<img src='http://ripnread.com/listen/EulerTotientFunction.svg'>
There are 
7 members to the 5-limit diamond, 
13 to the 7-limit diamond, 
19 to the 9-limit diamond, 
29 to the 11-limit diamond, 
41 to the 13-limit diamond, and 
49 to the 15-limit diamond; 
these suffice for most purposes. <-- the editorial comment that got me working


In [ ]:
# brute force method

for limit in (5, 7, 9, 11, 13, 15, 31):
    end_denom = limit + 1
    start_denom = (end_denom) // 2
    # print(f'{limit = }, {start_denom = }, {end_denom = }')
    o_numerator = np.arange(start_denom, end_denom,1)
    u_denominator = np.arange(start_denom, end_denom, 1)
    print(f'Tonality Diamond in the Cassandra orientation to the {limit}-limit', end='')
    all_ratios = []
    for oton_root in u_denominator:
        print()
        for overtone in o_numerator:
            if overtone < oton_root: oton = overtone * 2
            else: oton = overtone
            print(f'{str(Fraction(oton / oton_root).limit_denominator())}', end = '\t')
            all_ratios.append(oton / oton_root)

    all_ratios = list(dict.fromkeys(all_ratios)) # eliminate duplicates
    print(f'\nThere are {len(all_ratios)} unique members in the {limit}-limit diamond\n\n')      

In [ ]:
print(f'{str(Fraction(max(all_ratios)).limit_denominator())}') # what's the largest ratio in the set
print(f'{str(Fraction(min(all_ratios)).limit_denominator())}') # what's the smallest ratio

In [ ]:
# conversion between ratios (just) and cents (1200 equal per octave)
# ratios to cents:
ratio = 3/2
print(f'convert the ratio {str(Fraction(ratio).limit_denominator(max_denominator = 100))} to cents: {round(1200 * np.log(ratio)/np.log(2),1)}')
# cents to ratio
cents = 702
print(f'convert {cents} cents to a ratio: {str(Fraction(np.power(2, cents/1200)).limit_denominator(max_denominator = 100))}')

In [ ]:
# these are the values for steps using csound cpspch conversion. These numbers are in cents / 10000
csound_steps = np.array([0.0016727, 0.0033617, 0.0054964, 0.0056767, 0.0058692, 0.0060751, 0.0062961, 0.0065337, 0.0067900, 0.0070672, 
0.0073681, 0.0076956, 0.0080537, 0.0084467, 0.0088801, 0.0093603, 0.0098955, 0.0104955, 0.0111731, 0.0115458, 
0.0119443, 0.0124712, 0.0128298, 0.0133248, 0.0138573, 0.0144353, 0.0150637, 0.0157493, 0.0159920, 0.0165004, 
0.0170424, 0.0173268, 0.0176210, 0.0182404, 0.0189050, 0.0192558, 0.0196198, 0.0203910, 0.0212253, 0.0216687, 
0.0221309, 0.0241174, 0.0249171, 0.0241961, 0.0247741, 0.0253805, 0.0256950, 0.0258874, 0.0266871, 0.0275378, 
0.0277591, 0.0281358, 0.0289210, 0.0294135, 0.0297513, 0.0301847, 0.0304508, 0.0315641, 0.0327622, 0.0330761, 
0.0336130, 0.0340552, 0.0347408, 0.0352477, 0.0354547, 0.0359472, 0.0365825, 0.0369747, 0.0372408, 0.0374333, 
0.0386314, 0.0399090, 0.0401303, 0.0404442, 0.0409244, 0.0417508, 0.0424364, 0.0427373, 0.0435084, 0.0441278, 
0.0443081, 0.0446363, 0.0454214, 0.0459994, 0.0464428, 0.0467936, 0.0470781, 0.0475114, 0.0478259, 0.0498045, 
0.0516761, 0.0519551, 0.0524319, 0.0525745, 0.0528687, 0.0532428, 0.0536951, 0.0543015, 0.0551318, 0.0556737, 
0.0558796, 0.0563382, 0.0568717, 0.0571726, 0.0582512, 0.0591648, 0.0593718, 0.0597000, 0.0603000, 0.0606282, 
0.0608352, 0.0617488, 0.0628274, 0.0631283, 0.0636618, 0.0641204, 0.0643263, 0.0648682, 0.0656985, 0.0663049, 
0.0667672, 0.0671313, 0.0674255, 0.0676681, 0.0680449, 0.0683249, 0.0701955, 0.0721741, 0.0724886, 0.0729219, 
0.0732064, 0.0735572, 0.0740006, 0.0745786, 0.0753637, 0.0756919, 0.0758722, 0.0764916, 0.0772627, 0.0775636, 
0.0782492, 0.0790756, 0.0795558, 0.0798697, 0.0800910, 0.0813686, 0.0825667, 0.0827592, 0.0830253, 0.0834175, 
0.0840528, 0.0845453, 0.0847524, 0.0852592, 0.0859448, 0.0863870, 0.0869249, 0.0872478, 0.0884359, 0.0895492, 
0.0898153, 0.0902487, 0.0905865, 0.0910790, 0.0918642, 0.0922409, 0.0924622, 0.0933129, 0.0941126, 0.0943050, 
0.0946195, 0.0952259, 0.0958039, 0.0960829, 0.0968826, 0.0978691, 0.0983313, 0.0987747, 0.0996090, 0.1003802, 
0.1007442, 0.1010950, 0.1017596, 0.1024790, 0.1026732, 0.1029577, 0.1034996, 0.1040080, 0.1042507, 0.1049363, 
0.1055647, 0.1061427, 0.1066762, 0.1071702, 0.1076288, 0.1080557, 0.1084542, 0.1088269, 0.1095045, 0.1101045, 
0.1106397, 0.1111199, 0.1115533, 0.1119463, 0.1124044, 0.1126319, 0.1129328, 0.1132100, 0.1134663, 0.1137039, 
0.1139249, 0.1141308, 0.1143243, 0.1145036, 0.1166383, 0.1183273, 0.1200000])

In [ ]:
cent_array = csound_steps * 10_000
print(cent_array[:2]) # bottom two ratios in the 217 note per octave scale
print(cent_array[215:]) # top two ratios

In [ ]:
# convert from cpspch to ratios. start out with the csound value for cpspch
# show us the four values inserted in to the pitch table that were not in the tonality diamond.
print(f'{[str(Fraction(np.power(2, step / 1200)).limit_denominator(max_denominator = 200)) for step in cent_array[0:2]]}') 
print(f'{[str(Fraction(np.power(2, step / 1200)).limit_denominator(max_denominator = 200)) for step in cent_array[214:216]]}') 

In [ ]:
print(f'The ratios for the steps in the {len(cent_array)} step per octave scale:')
#                      +-- calculate the ratio by raising 2 to the power of (cents / 1200) 
#                      |                              +-- limit the denominator to a maximum of max_denominator
#                      |                              |                                                +-- array in csound * 10000
#                      |                              |                                                |
print(f'{[str(Fraction(np.power(2, step / 1200)).limit_denominator(max_denominator = 50)) for step in cent_array]}') 